In [ ]:

# Import necessary libraries
import os
import numpy as np
import pandas as pd
import pinecone
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from dotenv import load_dotenv


In [ ]:
# Load environment variables
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_ENV = os.getenv("PINECONE_ENV")